In [1]:
import numpy as np
import pandas as pd
import json
from flask import Flask, jsonify
from flask_cors import CORS
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as XGB

In [2]:
# Read in the tuned hyperparameters
try:
    # Open the JSON file
    with open('Models.json', 'r') as file:
        # Load the models
        models = json.load(file)
    # File is successfully opened and loaded
    print(models)
except FileNotFoundError:
    # Handle the FileNotFoundError
    print("The file 'Models.json' does not exist or cannot be opened.")

[[{'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}, {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1000, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 1000, 'penalty': 'l2', 'solver': 'liblinear'}], [{'booster': 'gbtree', 'learning_rate': 0.05, 'n_estimators': 100, 'objective': 'multi:softmax', 'eval_metric': 'logloss', 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'lambda': None, 'alpha': None, 'num_class': 2}, {'booster': 'gbtree', 'learning_rate': 0.05, 'n_estimators': 100, 'objective': 'multi:softmax', 'eval_metric': 'logloss', 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'lambda': None, 'alpha': None, 'num_class': 2}, {'booster': 'gbtree', 'learning_rate': 0.05, 'n_estimators': 100, 'objective': 'multi:softmax', 'eval_metric': 'logloss', 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'lambda': None, 'alpha': None, 'num_class': 2},

In [3]:
# Read in the list of columns that were not used to train
try:
    # Open the JSON file
    with open('FeatureDropped.json', 'r') as file:
        # Load the models
        features_dropped = json.load(file)
    # File is successfully opened and loaded
    print(features_dropped)
except FileNotFoundError:
    # Handle the FileNotFoundError
    print("The file 'FeatureDropped.json' does not exist or cannot be opened.")

['CustomerID', 'Gender', 'ChurnDate', 'FirstPersona', 'SecondPersona', 'ThirdPersona', 'CombinedPersonas', 'CurrLifecycle', 'SocialInfluencer', 'CurrLifecycle_Active', 'CurrLifecycle_Reactivated', 'CurrLifecycle_Churned', 'CurrLifecycle_Dormant', 'PrevLifecycle_Active', 'PrevLifecycle_Churned', 'PrevLifecycle_Dormant', 'PrevLifecycle_Reactivated', 'PrevLifecycle', 'CustomerId']


In [8]:
# Read in the train and test data which are the original customer base split into train and test
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Read in the oversampled train data for the fitting of models
# The oversampling is done from train.csv
#train_data_oversampled = pd.read_csv('train_oversampled.csv')
train_data_oversampled = pd.read_csv('train.csv')
input_features = [column for column in train_data.columns if column not in features_dropped]

In [9]:
# Encode categorical variables for logistic regression
label_encoder = LabelEncoder()
train_data_encoded = train_data.copy()
test_data_encoded = test_data.copy()
train_data_oversampled_encoded = train_data_oversampled.copy()

train_data_encoded['MonthsInactive'] = train_data_encoded['MonthsInactive'].fillna(0)
test_data_encoded['MonthsInactive'] = test_data_encoded['MonthsInactive'].fillna(0)
train_data_oversampled_encoded['MonthsInactive'] = train_data_oversampled_encoded['MonthsInactive'].fillna(0)

for column in train_data_encoded.columns:
    if train_data_encoded[column].dtype == 'object':
        train_data_encoded[column] = label_encoder.fit_transform(train_data_encoded[column])
        test_data_encoded[column] = label_encoder.fit_transform(test_data_encoded[column])
        train_data_oversampled_encoded[column] = label_encoder.fit_transform(train_data_oversampled_encoded[column])

In [10]:
# Copy data sets to do encoding and scaling for logistic regression
train_lgr = train_data_encoded.copy()
test_lgr = test_data_encoded.copy()
train_oversampled_lgr = train_data_oversampled_encoded.copy()

In [11]:
# Scale dataset (except for the labels) for logistic regression
scaler = StandardScaler()

#label_columns = ['CurrLifecycle_Active', 'CurrLifecycle_Reactivated', 'CurrLifecycle_Dormant', 'CurrLifecycle_Churned']
#feature_columns = [column for column in train_lgr.columns if column not in label_columns]

train_lgr_features = train_lgr[input_features]
#train_lgr_labels = train_lgr[label_columns]

train_lgr_features = pd.DataFrame(scaler.fit_transform(train_lgr_features), columns = train_lgr_features.columns)
#train_lgr = pd.concat([train_lgr_features, train_lgr_labels], axis = 1)

test_lgr_features = test_lgr[input_features]
#test_lgr_labels = test_lgr[label_columns]

test_lgr_features = pd.DataFrame(scaler.fit_transform(test_lgr_features), columns = test_lgr_features.columns)
#test_lgr = pd.concat([test_lgr_features, test_lgr_labels], axis = 1)

train_oversampled_lgr_features = train_oversampled_lgr[input_features]
#train_oversampled_lgr_labels = train_oversampled_lgr[label_columns]

train_oversampled_lgr_features = pd.DataFrame(scaler.fit_transform(train_oversampled_lgr_features), columns = train_oversampled_lgr_features.columns)
#train_oversampled_lgr = pd.concat([train_oversampled_lgr_features, train_oversampled_lgr_labels], axis = 1)

In [12]:
# Separate the lgr, xgb, and rf parameters into
# a list of 4 dictionaries with key value pairs of hyperparamters for Active, Reactivated, Dormant, and Churned Classification
lgr = models[0]
xgb = models[1]
rf = models[2]

In [13]:
y_train_oversampled_all_lgr = [train_oversampled_lgr['CurrLifecycle_Active'], train_oversampled_lgr['CurrLifecycle_Reactivated'], train_oversampled_lgr['CurrLifecycle_Dormant'], train_oversampled_lgr['CurrLifecycle_Churned']]
#y_test_encoded_all_lgr = [test_lgr['CurrLifecycle_Active'], test_lgr['CurrLifecycle_Reactivated'], test_lgr['CurrLifecycle_Dormant'], test_lgr['CurrLifecycle_Churned']]

y_train_oversampled_all = [train_data_oversampled_encoded['CurrLifecycle_Active'], train_data_oversampled_encoded['CurrLifecycle_Reactivated'], train_data_oversampled_encoded['CurrLifecycle_Dormant'], train_data_oversampled_encoded['CurrLifecycle_Churned']]
#y_test_all = [test_data_encoded['CurrLifecycle_Active'], test_data_encoded['CurrLifecycle_Reactivated'], test_data_encoded['CurrLifecycle_Dormant'], test_data_encoded['CurrLifecycle_Churned']]

In [28]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

In [29]:
lgr_models_list = []
xgb_models_list = []
rf_models_list = []
for i in range(4):
    # Train the lgr models and store in the list
    lgr_model = LogisticRegression(**lgr[i])
    X_train_resampled, y_train_resampled = smote.fit_resample(train_oversampled_lgr_features, y_train_oversampled_all_lgr[i])
    lgr_model.fit(X_train_resampled, y_train_resampled)
    #lgr_model.fit(train_oversampled_lgr_features, y_train_oversampled_all_lgr[i])
    lgr_models_list.append(lgr_model)
    
    # Train the xgb models and store in the list
    xgb_model = XGB.XGBClassifier(**xgb[i])
    X_train_resampled, y_train_resampled = smote.fit_resample(train_data_oversampled_encoded[input_features], y_train_oversampled_all[i])
    xgb_model.fit(X_train_resampled, y_train_resampled)
    #xgb_model.fit(train_data_oversampled_encoded[input_features], y_train_oversampled_all[i])
    xgb_models_list.append(xgb_model)
    
    # Train the rf models and store in the list
    rf_model = RandomForestClassifier(**rf[i])
    rf_model.fit(X_train_resampled, y_train_resampled)
    #rf_model.fit(train_data_oversampled_encoded[input_features], y_train_oversampled_all[i])
    rf_models_list.append(rf_model)

In [15]:
# Define the function to generate predictions from the trained models
def prediction(lgr, xgb, rf, data, features):
    # Generate predictions for data
    for i in range(len(lgr)):
        # lgr[0], xgb[0], rf[0] is the model with binary label Active/Non-Active
        if i == 0:
            data['lgr_Active_proba'] = lgr[0].predict_proba(data[features])[:, 1]
            data['xgb_Active_proba'] = xgb[0].predict_proba(data[features])[:, 1]
            data['rf_Active_proba'] = rf[0].predict_proba(data[features])[:, 1]
            
        # lgr[1], xgb[1], rf[1] is the model with binary label Reactivated/Non-Reactivated
        elif i == 1:
            data['lgr_Reactivated_proba'] = lgr[1].predict_proba(data[features])[:, 1]
            data['xgb_Reactivated_proba'] = xgb[1].predict_proba(data[features])[:, 1]
            data['rf_Reactivated_proba'] = rf[1].predict_proba(data[features])[:, 1]
            
        # lgr[2], xgb[2], rf[2] is the model with binary label Dormant/Non-Dormant
        elif i == 2:
            data['lgr_Dormant_proba'] = lgr[2].predict_proba(data[features])[:, 1]
            data['xgb_Dormant_proba'] = xgb[2].predict_proba(data[features])[:, 1]
            data['rf_Dormant_proba'] = rf[2].predict_proba(data[features])[:, 1]
           
        # lgr[3], xgb[3], rf[3] is the model with binary label Churned/Non-Churned
        elif i == 3:
            data['lgr_Churned_proba'] = lgr[3].predict_proba(data[features])[:, 1]
            data['xgb_Churned_proba'] = xgb[3].predict_proba(data[features])[:, 1]
            data['rf_Churned_proba'] = rf[3].predict_proba(data[features])[:, 1]
    
    # Calculate the average probability from the probabilities generated by each model (Ensemble Learning)
    data['average_Active_proba'] = data[['lgr_Active_proba', 'xgb_Active_proba', 'rf_Active_proba']].mean(axis = 1)
    data['average_Reactivated_proba'] = data[['lgr_Reactivated_proba', 'xgb_Reactivated_proba', 'rf_Reactivated_proba']].mean(axis = 1)
    data['average_Dormant_proba'] = data[['lgr_Dormant_proba', 'xgb_Dormant_proba', 'rf_Dormant_proba']].mean(axis = 1)
    data['average_Churned_proba'] = data[['lgr_Churned_proba', 'xgb_Churned_proba', 'rf_Churned_proba']].mean(axis = 1)
    
    # Based on the definition of lifecycle, it is not possible for a customer to have the below stated transitions
    # Active -> Reactivated, Dormant -> Dormant, Dormant -> Active, Reactivated -> Reactivated
    # Hence, set the probabilities of these cases to 0
    data['average_Active_proba'] = np.where((data['CurrLifecycle_Dormant'] == 1) & (data['average_Active_proba'] > 0), 0, data['average_Active_proba'])
    data['average_Reactivated_proba'] = np.where(((data['CurrLifecycle_Active'] == 1) | (data['CurrLifecycle_Reactivated'] == 1)) & (data['average_Reactivated_proba'] > 0), 0, data['average_Reactivated_proba'])
    data['average_Dormant_proba'] = np.where((data['CurrLifecycle_Dormant'] == 1) & (data['average_Dormant_proba'] > 0), 0, data['average_Dormant_proba'])
    
    # Normalize the probablities such that each row adds up to 1
    total_prob = data[['average_Active_proba', 'average_Reactivated_proba', 'average_Dormant_proba', 'average_Churned_proba']].sum(axis=1)
    data['average_Active_proba'] = data['average_Active_proba'] / total_prob
    data['average_Reactivated_proba'] = data['average_Reactivated_proba'] / total_prob
    data['average_Dormant_proba'] = data['average_Dormant_proba'] / total_prob
    data['average_Churned_proba'] = data['average_Churned_proba'] / total_prob
    
    # Rename the probabilities column
    data['Active'] = data['average_Active_proba']
    data['Reactivated'] = data['average_Reactivated_proba']
    data['Dormant'] = data['average_Dormant_proba']
    data['Churned'] = data['average_Churned_proba']
    
    # The lifecycle with the highest probability will be the predicted lifecycle
    data['PredictedLifecycle'] = data[['Active', 'Reactivated', 'Dormant', 'Churned']].idxmax(axis=1)

    return data

In [16]:
# Filter the existing active customers from the train_data
#train_drop_index = train_data_encoded[(train_data_encoded['CurrLifecycle_Dormant'] == 1) | (train_data_encoded['CurrLifecycle_Churned'] == 1)].index
#train_active = train_data_encoded.drop(train_drop_index)
train_active = train_data_encoded[train_data_encoded['CurrLifecycle_Churned'] != 1].copy()
#train_data = train_data.drop(train_drop_index)

# Returns the df with predictions for train_active (This train data only consists of existing active customers)
# Note: train_active_prediction is the encoded dataframe
train_active_prediction = prediction(lgr_models_list, xgb_models_list, rf_models_list, train_active, input_features)

# Returns the df with predictions for test_data 
# This test data consists of both active and churned customers to generate classification report for model performance evaluation
# Note: test_prediction is the encoded dataframe
test_prediction = prediction(lgr_models_list, xgb_models_list, rf_models_list, test_data_encoded, input_features)

In [17]:
# Add the probabilities and predicted Lifecycle columns to the non-encoded dataframe
train_data = pd.concat([train_data, train_active_prediction.loc[:, 'average_Active_proba':'average_Churned_proba'], train_active_prediction['PredictedLifecycle']], axis=1)
test_data = pd.concat([test_data, test_prediction.loc[:, 'average_Active_proba':'average_Churned_proba'], test_prediction['PredictedLifecycle']], axis=1)

In [18]:
# Encode the Predicted Lifecycles
test_data['PredLifecycle_Active'] = (test_data['PredictedLifecycle'] == 'Active').astype(int)
test_data['PredLifecycle_Reactivated'] = (test_data['PredictedLifecycle'] == 'Reactivated').astype(int)
test_data['PredLifecycle_Dormant'] = (test_data['PredictedLifecycle'] == 'Dormant').astype(int)
test_data['PredLifecycle_Churned'] = (test_data['PredictedLifecycle'] == 'Churned').astype(int)

train_data['PredLifecycle_Active'] = (train_data['PredictedLifecycle'] == 'Active').astype(int)
train_data['PredLifecycle_Reactivated'] = (train_data['PredictedLifecycle'] == 'Reactivated').astype(int)
train_data['PredLifecycle_Dormant'] = (train_data['PredictedLifecycle'] == 'Dormant').astype(int)
train_data['PredLifecycle_Churned'] = (train_data['PredictedLifecycle'] == 'Churned').astype(int)

In [19]:
# Generate classification report using the test predictions
report_table = classification_report(test_data['CurrLifecycle_Churned'], test_data['PredLifecycle_Churned']) 
report_dict = classification_report(test_data['CurrLifecycle_Churned'], test_data['PredLifecycle_Churned'], output_dict=True) 

In [20]:
print(report_table)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     26023
           1       0.93      1.00      0.96      6984

    accuracy                           0.98     33007
   macro avg       0.96      0.99      0.98     33007
weighted avg       0.98      0.98      0.98     33007



In [21]:
# Check format in dictionary
print(report_dict)

{'0': {'precision': 1.0, 'recall': 0.9786342850555279, 'f1-score': 0.9892017867547097, 'support': 26023.0}, '1': {'precision': 0.9262599469496021, 'recall': 1.0, 'f1-score': 0.9617185348388873, 'support': 6984.0}, 'accuracy': 0.9831550883146, 'macro avg': {'precision': 0.9631299734748011, 'recall': 0.9893171425277639, 'f1-score': 0.9754601607967985, 'support': 33007.0}, 'weighted avg': {'precision': 0.9843972329959106, 'recall': 0.9831550883146, 'f1-score': 0.9833865647902749, 'support': 33007.0}}


In [22]:
# Save classification report as json
json_data = json.dumps(report_dict)

with open("Report_Dict.json", "w") as json_file:
    json_file.write(json_data)

In [23]:
# Filter out the existing active customers from the test data
#test_drop_index = test_data[(test_data['CurrLifecycle_Dormant'] == 1) | (test_data['CurrLifecycle_Churned'] == 1)].index
#test_active_prediction = test_data.drop(test_drop_index) 

In [24]:
# Concat the train_data and test_data data
predicted_data = pd.concat([train_data, test_data], ignore_index=True)   

In [25]:
predicted_data.head()

,Gender,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Exited,CustomerId,ChurnDate,MonthsInactive,...,CurrLifecycle_Reactivated,average_Active_proba,average_Reactivated_proba,average_Dormant_proba,average_Churned_proba,PredictedLifecycle,PredLifecycle_Active,PredLifecycle_Reactivated,PredLifecycle_Dormant,PredLifecycle_Churned
0,Female,69,17,128651.25,2,12788,0,106009,NaN,0.0,...,0,0.992847,0.000000,0.005730,0.001422,Active,1,0,0,0
1,Male,35,11,91578.11,1,7335,0,62843,NaN,0.0,...,1,0.995137,0.000000,0.002447,0.002417,Active,1,0,0,0
2,Male,33,5,81619.90,2,4671,0,146611,NaN,3.0,...,0,0.000000,0.996713,0.000000,0.003287,Reactivated,0,1,0,0
3,Male,41,13,139825.30,2,11741,0,21915,NaN,2.0,...,0,0.000000,0.500209,0.000000,0.499791,Reactivated,0,1,0,0
4,Male,34,4,122105.11,2,4488,0,38342,NaN,0.0,...,0,0.997155,0.000000,0.001431,0.001414,Active,1,0,0,0


In [26]:
# Save it as csv for the API file to read
predicted_data.to_csv("Predicted_Data.csv")

In [27]:
# Check format of dictionary
predicted_data.head().to_dict(orient='records')

[{'Gender': 'Female',
  'Age': 69,
  'Tenure': 17,
  'Balance': 128651.25,
  'NumOfProducts': 2,
  'EstimatedSalary': 12788,
  'Exited': 0,
  'CustomerId': 106009,
  'ChurnDate': nan,
  'MonthsInactive': 0.0,
  'TransactionFreq': 28,
  'TransactionAmt': 5293.38,
  'ServiceSupportFrequency': 7,
  'NPS': 8.0,
  'Education': 'Post-Doc',
  'EmploymentStatus': 'Self-employed',
  'MaritalStatus': 'Married',
  'HousingStatus': 'rented',
  'Dependants': 5,
  'MarketingOffersAcceptance': 0.1130881326031034,
  'PaymentMethod': 'Paynow',
  'BrandSatisfaction': 4,
  'FeatureSatisfaction': 2,
  'SupportSatisfaction': 2,
  'FeatureSupportFrequency': 5,
  'LoanAmt': 20165.6,
  'IncomeSource': 'Self-employment',
  'Retention': 0.1330267586850883,
  'ChangeInBehaviourMkt': 0.688558928001504,
  'ChangeInBehaviourCust': 1.0577501226273818,
  'PrevLifecycle': 'Reactivated',
  'CurrLifecycle': 'Active',
  'Happiness': 1,
  'SocialInfluencer': 1,
  'Savings Savant': 44.5,
  'Digital Dynamos': 37.0,
  'Trust